In [1]:
import nfl_data_py as nfl
import pandas as pd
from NFL_Player import Player
from Injury import *
import requests
from bs4 import BeautifulSoup
from helperFunctions import *
from googlesearch import search
import re
from datetime import datetime
import concurrent.futures
from tqdm import tqdm
import time
import injury_analysis

In [2]:
# nfl.import_injuries([2020])["full_name"]
nflInjuries20 = nfl.import_injuries([2020])
nflInjuries21 = nfl.import_injuries([2021])
nflInjuries22 = nfl.import_injuries([2022])
nflInjuries23 = nfl.import_injuries([2023])
nflInjuries24 = nfl.import_injuries([2024])
allNFLInjuries = pd.concat([nflInjuries20, nflInjuries21, nflInjuries22, nflInjuries23, nflInjuries24])

In [3]:
historicalNFLInjuries = allNFLInjuries.loc[(allNFLInjuries["season"] != 2024.0) | (allNFLInjuries["week"] != 4.0)].copy()
historicalNFLInjuries['InjuredInGame?'] = historicalNFLInjuries['practice_status'].apply(lambda x: x.strip() == "")


In [ ]:
def filterHistoricalNFLInjuries(injuries_df, season=None, game_type=None, team=None, week=None, position=None, full_name=None,last_name=None, first_name=None, report_primary_injury=None, report_secondary_injury=None, report_status=None,
                                practice_primary_injury=None, practice_secondary_injury=None, practice_status=None, injured_in_game = None):
    injuries_df_copied = injuries_df.copy()
    if season is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["season"] == season]
    if game_type is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["game_type"] == game_type]
    if team is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["team"] == team]
    if week is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["week"] == week]
    if position is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["position"] == position]
    if full_name is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["full_name"] == full_name]
    if first_name is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["first_name"] == first_name]
    if last_name is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["last_name"] == last_name]
    if report_primary_injury is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["report_primary_injury"] == report_primary_injury]
    if report_secondary_injury is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["report_secondary_injury"] == report_secondary_injury]
    if report_status is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["report_status"] == report_status]
    if practice_primary_injury is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["practice_primary_injury"] == practice_primary_injury]
    if practice_secondary_injury is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["practice_secondary_injury"] == practice_secondary_injury]
    if practice_status is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["practice_status"] == practice_status]
    if injured_in_game is not None:
        injuries_df_copied = injuries_df_copied[injuries_df_copied["InjuredInGame?"] == injured_in_game]
    

In [ ]:
# from helperFunctions import *
x = filterHistoricalNFLInjuries(historicalNFLInjuries, postion="WR")
x.head()

In [ ]:
historicalNFLInjuries
# historicalNFLInjuries[historicalNFLInjuries["gsis_id"] == "00-0039151"]

In [ ]:

# {('Emmanuel Forbes', 'CB'): Player}}
player_names = {}
players = []
for index, row in historicalNFLInjuries.iterrows():
    keysList = list(player_names.keys())
    temp_injury = GameInjuryReport(
        row["season"], 
        row["game_type"], 
        row["team"], 
        row["week"], 
        row["report_primary_injury"], 
        row["report_secondary_injury"], 
        row["report_status"], 
        row["practice_primary_injury"], 
        row["practice_secondary_injury"], 
        row["practice_status"], 
        row["InjuredInGame?"]
    )

    if row["full_name"] not in keysList:
        player_names[row["full_name"]] = Player(row["first_name"], row["last_name"], row["position"])
    player_names.get(row["full_name"]).add_injury(temp_injury)
        
for x in player_names.get("Emmanuel Forbes").injuries:
    print(x.display_report())

In [ ]:
# DeAndre	Hopkins	
for x in player_names.get("DeAndre Hopkins").injuries:
    print(x.display_report())

In [ ]:
for x in player_names.keys():
    print(x)

In [ ]:
from googlesearch import search
import re
import requests
from bs4 import BeautifulSoup

def get_draft_sharks_url(first_name, last_name):
    full_name = first_name + " " + last_name
    search_query = f"{full_name} Draft Sharks"
    first_name_clean = ''.join(char for char in first_name if char.isalnum()).lower()
    last_name_clean = ''.join(char for char in last_name if char.isalnum()).lower()
    # Perform a Google search and get the URLs
    for url in search(search_query, num_results=10):
        if 'draftsharks.com' in url:
            if first_name_clean+"-"+last_name_clean in url:
                if 'injury-history' in url:
                    return url
                match = re.search(r'/(\d+)$', url)
                if match:
                    return "https://www.draftsharks.com/fantasy/injury-history/"+first_name_clean+"-"+last_name_clean+"/"+(match.group(1))    
    return None

for x in player_names.keys():
    url = get_draft_sharks_url(player_names[x].first_name, player_names[x].last_name)
    full_name = player_names[x].first_name+" "+ player_names[x].last_name
    if url:
        print(f"Draft Sharks URL for {full_name}: {url}")
    else:
        print(f"Invalid Draft Sharks found for {full_name}.") 


# for index, row in historicalNFLInjuries.iterrows():
#     url = get_draft_sharks_url(row["first_name"], row["last_name"])
#     full_name = row["full_name"]
#     if url:
#         try:
#             response = requests.get(url)
#             soup = BeautifulSoup(response.content, 'html.parser')
#             table_rows = soup.find("div",class_="injury-history-table-container").find("tbody").findAll("tr")
#             for row in table_rows:
#                 row_data = row.findAll("td")
#                 injury_date = row_data[0] # Date
#                 injury_league = row_data[1] # League
#                 injury = row_data[2] # Injury
#                 injury_details = row_data[3] # Description
                
#             print(f"Draft Sharks URL for {full_name}: {url}")
#         except:
#             print(f"Invalid Draft Sharks found for {full_name}.")

#     else:
#         print(f"No Draft Sharks URL found for {full_name}.")


In [4]:
historicalNFLInjuries2 = historicalNFLInjuries.copy()
historicalNFLInjuries2['combined_injury_description'] = (
    historicalNFLInjuries2['season'].fillna('').astype(str) + " " +
    historicalNFLInjuries2['game_type'].fillna('').astype(str) + " " +
    historicalNFLInjuries2['team'].fillna('').astype(str) + " " +
    historicalNFLInjuries2['week'].fillna('').astype(str) + " " +
    historicalNFLInjuries2['position'].fillna('').astype(str) + " " +
    historicalNFLInjuries2['full_name'].fillna('').astype(str) + " " +
    historicalNFLInjuries2['first_name'].fillna('').astype(str) + " " +
    historicalNFLInjuries2['last_name'].fillna('').astype(str) + " " +
    historicalNFLInjuries2['report_primary_injury'].fillna('').astype(str) + " " +
    historicalNFLInjuries2['report_secondary_injury'].fillna('').astype(str) + " " +
    historicalNFLInjuries2['report_status'].fillna('').astype(str) + " " +
    historicalNFLInjuries2['practice_primary_injury'].fillna('').astype(str) + " " +
    historicalNFLInjuries2['practice_secondary_injury'].fillna('').astype(str) + " " +
    historicalNFLInjuries2['practice_status'].fillna('').astype(str) + " " +
    historicalNFLInjuries2['InjuredInGame?'].fillna('').astype(str)
)


In [2]:
sentiment_analyzer = injury_analysis.pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
)

: 